[大图](TF1.html), [下载](origin_files/TF1.ipynb)

In [1]:
import tensorflow as tf

In [2]:
hello=tf.constant('hello!')
sess=tf.Session()#创建一个session
hey=sess.run(hello)#使用session来运行
# sess.close()#关闭session，释放资源

batch归一化

In [ ]:
h1=tf.contrib.layers.fully_connected(x,100,activation_fn=None,scope='bn')
h2=tf.contrib.layers.batch_norm(h1,center=True,scale=True,is_training=phase,scope='bn')

# iris为例的一个完整的神经网络训练过程

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
sess=tf.Session()

- 加载数据并dummy

In [2]:
from sklearn import datasets
dataset=datasets.load_iris()
data=dataset.data
target=dataset.target
X=data
target_df=pd.Series(target)
target_df=pd.get_dummies(target_df)
target=target_df.values
from sklearn import model_selection
train_data,test_data,train_target,test_target=model_selection.train_test_split(
    data,target,test_size=0.2,train_size=0.8,random_state=12345)#划分训练集和测试集

- 基本参数

In [3]:
X=train_data
Y=train_target
dataset_size=data.shape[0]
batch_size=10

- 定义张量

In [4]:
w1=tf.Variable(tf.truncated_normal(shape=(4,10),stddev=1),name='weight1')
b1=tf.Variable(tf.truncated_normal(shape=(1,10),stddev=1),name='bias1')
w2=tf.Variable(tf.truncated_normal(shape=(10,3),stddev=1),name='weight2')
b2=tf.Variable(tf.truncated_normal(shape=(1,3),stddev=1),name='bias2')
x=tf.placeholder(shape=(None,4),dtype=tf.float32)
y_=tf.placeholder(shape=(None,3),dtype=tf.float32)

- 信号正向传播

In [5]:
a=tf.nn.sigmoid(tf.matmul(x,w1)+b1)
y=tf.nn.softmax(tf.matmul(a,w2)+b2)

- 误差与优化

In [6]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cross_entropy)

- 迭代学习

In [7]:
sess.run(tf.global_variables_initializer())#全部初始化, 千万别忘了！

STEPS=50000
for i in range(STEPS):
    start=(i*batch_size)%batch_size
    end=min(start+batch_size,dataset_size)
    sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
    if i%5000==0:
        total_cross_entropy=sess.run(cross_entropy,feed_dict={x:X,y_:Y})
        print(i,'training step,cross entropy is',total_cross_entropy)

0 training step,cross entropy is 1.7206
5000 training step,cross entropy is 0.744731
10000 training step,cross entropy is 0.577951
15000 training step,cross entropy is 0.49825
20000 training step,cross entropy is 0.435902
25000 training step,cross entropy is 0.380805
30000 training step,cross entropy is 0.33211
35000 training step,cross entropy is 0.291082
40000 training step,cross entropy is 0.257882
45000 training step,cross entropy is 0.231364


- predict

In [8]:
y_train_predict=sess.run(y,feed_dict={x:train_data})
y_test_predict=sess.run(y,feed_dict={x:test_data})
# sess.close()

- 模型评价

In [9]:
import sklearn.metrics as metrics
print('train',metrics.confusion_matrix(train_target.argmax(axis=1),y_train_predict.argmax(axis=1)))
print('test',metrics.confusion_matrix(test_target.argmax(axis=1),y_test_predict.argmax(axis=1)))

train [[39  0  0]
 [ 0 34  4]
 [ 0  0 43]]
test [[11  0  0]
 [ 0 10  2]
 [ 0  0  7]]


In [10]:
correct_prediction=tf.equal(tf.argmax(y,axis=1),tf.argmax(y_,axis=1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
sess.run(accuracy,feed_dict={x:train_data,y_:train_target})

0.96666664

In [ ]:
tf.GraphKeys.GLOBAL_VARIABLES #所有变量
tf.GraphKeys.TRAINABLE_VARIABLES #所有可训练变量
tf.GraphKeys.SUMMARIES #日志相关张量
tf.GraphKeys.QUEUE_RUNNERS #处理输入的QueueRunner
tf.GraphKeys.MOVING_AVERAGE_VARIABLES #所有计算了移动平均值的变量

# 其它代码

In [ ]:
with tf.Graph().as_default():
    pass


In [ ]:
import math
# 参数初始化
# 均值0，方差1的正态随机数：
weights=tf.Variable(tf.truncated_normal([3,2],stddev=1),name='weights')
#正态随机数除以神经元的个数
weights=tf.Variable(tf.truncated_normal([500,25],stddev=1/math.sqrt(500)),name='weights')

In [ ]:
# DropOut
# 每一轮训练临时性的关闭一些节点
# 降低VC维数量，减少过拟合风险

keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

# slim
很有用的东西，可以不用设置中间变量，给函数赋值

In [6]:
slim=tf.contrib.slim
with slim.arg_scope([slim.conv2d,slim.fully_connected],weights_regularizer=slim.l2_regularizer(weights_decay)):
    pass
